In [14]:
import pandas as pd
import numpy as np
import ast
import pickle

In [ ]:
#pip install wikipedia-api

In [12]:
import wikipediaapi


wiki_wiki = wikipediaapi.Wikipedia(
    user_agent='MyProjectName',
        language='en',
        extract_format=wikipediaapi.ExtractFormat.WIKI
)

#  Get data for each category

In [3]:
from random import shuffle


def url_categorymembers(categorymembers,list_pages=[], level=0, max_level=3, i=0):
    pages_cat = list(categorymembers.values()) #toutes les pages de la catégorie
    shuffle(pages_cat)

    for c in pages_cat:
        if i==1000: #1000 pages par catégorie
            break

        if c.title[:6]!="Portal" and c.title[:5]!="File:" and c.title[:8]!="Category": # vérifie que c'est bien un article
            i+=1
            d = {}
            d["titre"] = c.title
            d["contenu"] = c.text
            d["liens"] = list(c.links.keys())
            list_pages.append(d)

        if c.ns == wikipediaapi.Namespace.CATEGORY and level < max_level: # descend d'un niveau
            i = url_categorymembers(c.categorymembers, list_pages, level=level + 1, max_level=max_level, i=i)[1]

    return list_pages, i

In [ ]:
cat_arts = wiki_wiki.page("Category:The arts") # Arts
arts, _ = url_categorymembers(cat_arts.categorymembers)
pd.DataFrame(arts).to_csv('arts.csv')

In [ ]:
cat_games = wiki_wiki.page("Category:Games") # Games
games, _= url_categorymembers(cat_games.categorymembers)

pd.DataFrame(games).to_csv('games.csv')

In [ ]:
cat_youth = wiki_wiki.page("Category:Youth") # Kids and Teens (not exact)
youth, _ = url_categorymembers(cat_youth.categorymembers)

pd.DataFrame(youth).to_csv('youth.csv')

In [ ]:
cat_reference = wiki_wiki.page("Category:Reference") # Reference
reference, _ = url_categorymembers(cat_reference.categorymembers)

pd.DataFrame(reference).to_csv('reference.csv')

In [ ]:
cat_shopping = wiki_wiki.page("Category:Retailing") # Shopping
shopping, _ = url_categorymembers(cat_shopping.categorymembers)

pd.DataFrame(shopping).to_csv('shopping.csv')

In [ ]:
cat_business = wiki_wiki.page("Category:Business") # Business
business, _ = url_categorymembers(cat_business.categorymembers)

pd.DataFrame(business).to_csv('business.csv')

In [ ]:
cat_health = wiki_wiki.page("Category:Health") # Health
health, _ = url_categorymembers(cat_health.categorymembers)

pd.DataFrame(health).to_csv('health.csv')

In [ ]:
cat_news = wiki_wiki.page("Category:News") # News
news, _ = url_categorymembers(cat_news.categorymembers)

pd.DataFrame(news).to_csv('news.csv')

In [ ]:
cat_geography = wiki_wiki.page("Category:Geography") # Regional (not exact)
geography, _ = url_categorymembers(cat_geography.categorymembers)

pd.DataFrame(geography).to_csv('geography.csv')

In [ ]:
cat_society = wiki_wiki.page("Category:Society") # Society
society, _ = url_categorymembers(cat_society.categorymembers)

pd.DataFrame(society).to_csv('society.csv')

In [ ]:
cat_computers = wiki_wiki.page("Category:Computers") # Computers
computers, _ = url_categorymembers(cat_computers.categorymembers)

pd.DataFrame(computers).to_csv('computers.csv')

In [ ]:
cat_home = wiki_wiki.page("Category:Home") # Home
home, _ = url_categorymembers(cat_home.categorymembers)

pd.DataFrame(home).to_csv('home.csv')

In [ ]:
cat_recreation = wiki_wiki.page("Category:Recreation") # Recreation
recreation, _ = url_categorymembers(cat_recreation.categorymembers)

pd.DataFrame(recreation).to_csv('recreation.csv')

In [ ]:
cat_science = wiki_wiki.page("Category:Science") # Science
science, _ = url_categorymembers(cat_science.categorymembers)

pd.DataFrame(science).to_csv('science.csv')

In [ ]:
cat_sports = wiki_wiki.page("Category:Sports") # Sports
sports, _ = url_categorymembers(cat_sports.categorymembers)

pd.DataFrame(sports).to_csv('sports.csv')

In [ ]:
cat_world = wiki_wiki.page("Category:World") # World
world, _ = url_categorymembers(cat_world.categorymembers)

pd.DataFrame(world).to_csv('world.csv')

In [53]:
csv_files = [
    'arts.csv', 'business.csv', 'computers.csv', 'games.csv',
    'geography.csv', 'health.csv', 'home.csv', 'news.csv',
    'recreation.csv', 'reference.csv', 'science.csv', 'shopping.csv',
    'society.csv', 'sports.csv', 'world.csv', 'youth.csv'
]

# Liste pour stocker les DataFrames
dataframes = []

# Lire chaque fichier CSV et l'ajouter à la liste des DataFrames
for file in csv_files:
    df = pd.read_csv(f'./Data/{file}')
    dataframes.append(df)

# Combiner tous les DataFrames en un seul
combined_df = pd.concat(dataframes, ignore_index=True)

# Écrire le DataFrame combiné dans un nouveau fichier CSV
combined_df.to_csv('./Data/all.csv', index=False)

# Compute Page Rank vectors (3.2)

## Matrices d'adjacence

Construction de la matrice sans utiliser les fichiers :

In [4]:
# build the adjacency matrix
def build_adjacency_matrix_2(df): #pas optimisé
    num_pages = df.shape[0]
    adjacency_matrix = np.zeros((num_pages, num_pages))
    df["liens"] = df["liens"].apply(ast.literal_eval)
    
    for index, row in df.iterrows():
        N = len(np.where(np.isin(row['liens'],df['titre'].values))[0]) # nombres de liens que fait la page
        for link in row['liens']:
            if link in df['titre'].values:
                j = np.where(df['titre'] == link )[0]
                adjacency_matrix[j,index] += 1/N

    return adjacency_matrix



def build_adjacency_matrix(df):
    df["liens"] = df["liens"].apply(ast.literal_eval)  # Convertir les liens de type string en listes

    # Créer un dictionnaire pour mapper les titres aux indices
    title_to_index = {title: i for i, title in enumerate(df['titre'])}
    
    # Initialiser la matrice d'adjacence avec des zéros
    num_pages = len(df)
    adjacency_matrix = np.zeros((num_pages, num_pages))
    
    # Construire une série pour mapper les liens aux indices
    link_series = pd.Series(df['liens'].values.tolist())
    link_indices = link_series.map(lambda x: [title_to_index[link] for link in x if link in title_to_index])
    
    # Compter le nombre de liens sortants pour chaque article
    df['num_liens'] = link_indices.apply(len)
    
    for i, indices in enumerate(link_indices):
        if len(indices)>0:
            adjacency_matrix[indices, i] = 1 / df.at[i, 'num_liens']
    
    return adjacency_matrix

In [108]:
matrice_all= build_adjacency_matrix(pd.read_csv('./Data/all.csv'))
np.save('./Matrices/matrice_all.npy', matrice_all)

Construction de la matrice en utilisant les fichiers :

In [ ]:
fichier_graphe = pd.read_csv('enwiki.wikilink_graph.2018-03-01.csv')

# build the adjacency matrix
def build_adjacency_matrix(topic_articles, inversed_index_articles):
    num_pages = len(topic_articles)
    adjacency_matrix = np.zeros((num_pages, num_pages))

    for i,titre in enumerate(topic_articles):
        #links = page_links(pages[i])
        links = fichier_graphe[fichier_graphe['page_title_from']==titre]
        for link in links:
            if link in topic_articles:
                j = topic_articles.index(link)
                adjacency_matrix[j][i] = 1

    return adjacency_matrix/sum(adjacency_matrix,axis=0)



## Calcul des vecteurs

In [13]:
# compute PageRank scores
def compute_pagerank(M, alpha=0.25, bias = None): # bias = None pour page_rank normal, bias = topic sinon
    # M = adjacency_matrix
    # damping_factor = 1-alpha

    N = M.shape[0]

    if bias is None : # page_rank normal, pas topic_sensitive
        p = np.ones(N) / N 

    else: # ODP-biasing 
        articles = pd.read_csv(f'./Data/all.csv')["titre"].values
        articles_cat = pd.read_csv(f'./Data/{bias}.csv')["titre"].values
        p = np.where(np.isin(articles,articles_cat), 1/len(articles_cat), 0)

    #M_prime = (1-alpha) * M + alpha * teleportation_matrix

    rank = np.ones(N) / N
    old_rank = np.zeros(N)

    epsilon = 1.0e-3
    max_iterations = 10
    iterations = 0

    while np.sum(np.abs(rank - old_rank)) > epsilon and iterations < max_iterations:
        old_rank = rank.copy()
        rank = (1-alpha)*M @ rank + p # equation 5
        #normaliser ?
        iterations += 1

    return rank

In [4]:
# Compute PageRank scores for each topic
import numpy as np

Topic_names = ["arts","games","youth","reference", "shopping", "business", "health", "news","geography","society","computers","home","recreation","science","sports","world"]

adjacency_matrix = np.load(f'./Matrices/matrice_all.npy')

topic_pagerank = {}
for topic in Topic_names:
    #adjacency_matrix = np.load(f'./Matrices/matrice_{topic}.npy')
    
    pagerank_scores = compute_pagerank(adjacency_matrix,bias=topic)

    print(f"Topic: {topic}")
    print(pagerank_scores[:5])

    topic_pagerank[topic] = pagerank_scores

    

NameError: name 'np' is not defined

In [114]:
with open('topic_pagerank.pkl', 'wb') as fichier:
    pickle.dump(topic_pagerank, fichier)

In [ ]:
adjacency_matrix = np.load(f'./Matrices/matrice_all.npy')
pagerank_normal = compute_pagerank(adjacency_matrix,bias=None)
with open('normal_pagerank.pkl', 'wb') as fichier:
    pickle.dump(pagerank_normal, fichier)

# Query-time importance score (3.3)

## Calcul de D

In [65]:
import json
from sklearn.feature_extraction.text import CountVectorizer


categories_text = {}
for c in Topic_names:
    liste_textes = pd.read_csv(f'./Data/{c}.csv')["contenu"].values.astype(str)

    texte = ' '.join(liste_textes)
    categories_text[c] = texte

D_categories = {}
terms_indexes_categories = {}

for c, text in categories_text.items():
    vectorizer = CountVectorizer(stop_words='english')
    D_categories[c] = vectorizer.fit_transform([text])
    terms_indexes_categories[c] = vectorizer.get_feature_names_out()
    

In [128]:
with open('D.pkl', 'wb') as fichier:
    pickle.dump(D_categories, fichier)

with open('terms_indexes.pkl', 'wb') as fichier:
    pickle.dump(terms_indexes_categories, fichier)

## Score

In [ ]:
!pip install python-terrier

In [7]:
# Indexation des articles 
import pandas as pd
import pyterrier as pt

if not pt.started():
  pt.init(boot_packages=["com.github.terrierteam:terrier-prf:-SNAPSHOT"])

df = pd.read_csv(f'./Data/all.csv')[["titre","contenu"]]
df = df.dropna()
df.columns = ["docno", "text"]

pd_indexer = pt.DFIndexer("./pd_index")
indexref = pd_indexer.index(df["text"], df["docno"])
index = pt.IndexFactory.of(indexref)

Topic_names = ["arts","games","youth","reference", "shopping", "business", "health", "news","geography","society","computers","home","recreation","science","sports","world"]

23:11:06.130 [main] WARN org.terrier.structures.indexing.Indexer - skipping null document
23:11:06.258 [main] WARN org.terrier.structures.indexing.Indexer - skipping null document


Traceback (most recent call last):
  File "jnius/jnius_proxy.pxi", line 50, in jnius.PythonJavaClass.invoke
  File "jnius/jnius_proxy.pxi", line 76, in jnius.PythonJavaClass._invoke
  File "/opt/homebrew/lib/python3.11/site-packages/pyterrier/index.py", line 778, in next
    lastdoc = self.convertFn(text, meta)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/homebrew/lib/python3.11/site-packages/pyterrier/index.py", line 687, in convertDoc
    return TaggedDocument(StringReader(text_row), hashmap, Tokeniser.getTokeniser())
                          ^^^^^^^^^^^^^^^^^^^^^^
  File "jnius/jnius_export_class.pxi", line 269, in jnius.JavaClass.__init__
  File "jnius/jnius_export_class.pxi", line 362, in jnius.JavaClass.call_constructor
  File "jnius/jnius_conversion.pxi", line 74, in jnius.populate_args
  File "jnius/jnius_utils.pxi", line 193, in jnius.check_assignable_from_str
TypeError: Invalid instance of 'java/lang/Float' passed for a 'java/lang/String'
Traceback (most recent call

23:11:06.335 [main] WARN org.terrier.structures.indexing.Indexer - skipping null document
23:11:06.348 [main] WARN org.terrier.structures.indexing.Indexer - skipping null document
23:11:06.356 [main] WARN org.terrier.structures.indexing.Indexer - skipping null document


Traceback (most recent call last):
  File "jnius/jnius_proxy.pxi", line 50, in jnius.PythonJavaClass.invoke
  File "jnius/jnius_proxy.pxi", line 76, in jnius.PythonJavaClass._invoke
  File "/opt/homebrew/lib/python3.11/site-packages/pyterrier/index.py", line 778, in next
    lastdoc = self.convertFn(text, meta)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/homebrew/lib/python3.11/site-packages/pyterrier/index.py", line 687, in convertDoc
    return TaggedDocument(StringReader(text_row), hashmap, Tokeniser.getTokeniser())
                          ^^^^^^^^^^^^^^^^^^^^^^
  File "jnius/jnius_export_class.pxi", line 269, in jnius.JavaClass.__init__
  File "jnius/jnius_export_class.pxi", line 362, in jnius.JavaClass.call_constructor
  File "jnius/jnius_conversion.pxi", line 74, in jnius.populate_args
  File "jnius/jnius_utils.pxi", line 193, in jnius.check_assignable_from_str
TypeError: Invalid instance of 'java/lang/Float' passed for a 'java/lang/String'
Traceback (most recent call

23:11:08.741 [main] WARN org.terrier.structures.indexing.Indexer - skipping null document


...

Traceback (most recent call last):
  File "jnius/jnius_proxy.pxi", line 50, in jnius.PythonJavaClass.invoke
  File "jnius/jnius_proxy.pxi", line 76, in jnius.PythonJavaClass._invoke
  File "/opt/homebrew/lib/python3.11/site-packages/pyterrier/index.py", line 778, in next
    lastdoc = self.convertFn(text, meta)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/homebrew/lib/python3.11/site-packages/pyterrier/index.py", line 687, in convertDoc
    return TaggedDocument(StringReader(text_row), hashmap, Tokeniser.getTokeniser())
                          ^^^^^^^^^^^^^^^^^^^^^^
  File "jnius/jnius_export_class.pxi", line 269, in jnius.JavaClass.__init__
  File "jnius/jnius_export_class.pxi", line 362, in jnius.JavaClass.call_constructor
  File "jnius/jnius_conversion.pxi", line 74, in jnius.populate_args
  File "jnius/jnius_utils.pxi", line 193, in jnius.check_assignable_from_str
TypeError: Invalid instance of 'java/lang/Float' passed for a 'java/lang/String'


23:11:18.876 [main] WARN org.terrier.structures.indexing.Indexer - skipping null document
23:11:18.951 [main] WARN org.terrier.structures.indexing.Indexer - skipping null document
23:11:18.990 [main] WARN org.terrier.structures.indexing.Indexer - skipping null document
23:11:18.996 [main] WARN org.terrier.structures.indexing.Indexer - skipping null document


Traceback (most recent call last):
  File "jnius/jnius_proxy.pxi", line 50, in jnius.PythonJavaClass.invoke
  File "jnius/jnius_proxy.pxi", line 76, in jnius.PythonJavaClass._invoke
  File "/opt/homebrew/lib/python3.11/site-packages/pyterrier/index.py", line 778, in next
    lastdoc = self.convertFn(text, meta)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/homebrew/lib/python3.11/site-packages/pyterrier/index.py", line 687, in convertDoc
    return TaggedDocument(StringReader(text_row), hashmap, Tokeniser.getTokeniser())
                          ^^^^^^^^^^^^^^^^^^^^^^
  File "jnius/jnius_export_class.pxi", line 269, in jnius.JavaClass.__init__
  File "jnius/jnius_export_class.pxi", line 362, in jnius.JavaClass.call_constructor
  File "jnius/jnius_conversion.pxi", line 74, in jnius.populate_args
  File "jnius/jnius_utils.pxi", line 193, in jnius.check_assignable_from_str
TypeError: Invalid instance of 'java/lang/Float' passed for a 'java/lang/String'
Traceback (most recent call

23:11:19.576 [main] WARN org.terrier.structures.indexing.Indexer - skipping null document
23:11:19.688 [main] WARN org.terrier.structures.indexing.Indexer - skipping null document


Traceback (most recent call last):
  File "jnius/jnius_proxy.pxi", line 50, in jnius.PythonJavaClass.invoke
  File "jnius/jnius_proxy.pxi", line 76, in jnius.PythonJavaClass._invoke
  File "/opt/homebrew/lib/python3.11/site-packages/pyterrier/index.py", line 778, in next
    lastdoc = self.convertFn(text, meta)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/homebrew/lib/python3.11/site-packages/pyterrier/index.py", line 687, in convertDoc
    return TaggedDocument(StringReader(text_row), hashmap, Tokeniser.getTokeniser())
                          ^^^^^^^^^^^^^^^^^^^^^^
  File "jnius/jnius_export_class.pxi", line 269, in jnius.JavaClass.__init__
  File "jnius/jnius_export_class.pxi", line 362, in jnius.JavaClass.call_constructor
  File "jnius/jnius_conversion.pxi", line 74, in jnius.populate_args
  File "jnius/jnius_utils.pxi", line 193, in jnius.check_assignable_from_str
TypeError: Invalid instance of 'java/lang/Float' passed for a 'java/lang/String'
Traceback (most recent call

In [25]:
df = pd.read_csv(f'./Data/all.csv')[["titre","contenu"]]
df = df.fillna('missing')

In [66]:
def sqd(q, topic_pagerank, D, terms_indexes, index):
    # Using a text index, recuperer les docs qui contiennent q
    br = pt.BatchRetrieve(index, wmodel="CoordinateMatch") # CoordinateMatch renvoie 1 si le terme est dans le doc, 0 sinon
    docs_retrieved = br.search(q) # Renvoie le nombre de termes de la query qui sont dans chaque doc
    docs_query = docs_retrieved[docs_retrieved["score"] == len(q.split())]  # Garder uniquement les docs qui contiennent tous les termes de la query
    docs_index = docs_query["docno"].values
    
    # calculer P(cj|q)
    probas_c_q = np.zeros((16))
    p_c = 1/16 # "The quantity P(cj) is not as straightforward. We chose to make it uniform"

    for i,topic in enumerate(Topic_names):
        indexes = np.in1d(terms_indexes[topic], np.array(q)).nonzero()[0]

        if len(indexes)==0:
            probas_c_q[i] = 0
        else:
            p_q_c = D[topic].tocsc()[indexes]/D[topic].tocsc().sum()
            p_q_c = p_q_c.toarray()
            probas_c_q[i] = p_c * np.product(p_q_c)
            
    df = pd.read_csv(f'./Data/all.csv')
    df.columns = ["id", "titre", "contenu", "liens"]
            
    docs = df[df["titre"].isin(docs_index)]
    
    # print(docs)
    
    docs["score"] = docs.apply(lambda x: sum([probas_c_q[Topic_names.index(topic)]*topic_pagerank[topic][x["id"]] for topic in Topic_names]), axis=1)
    
    # print(docs)

    # garder les 3 meilleurs cj
    # sorted_indices = np.argsort(-probas_c_q)
    # 
    # top_indices = sorted_indices[:3]
    # top_values = probas_c_q[top_indices]
    # 
    # # somme des P(cj|q) * rank j
    # res = 0
    # for i in range(3):
    #     res+= top_values[i]*topic_pagerank[Topic_names[top_indices[i]]]
    #return np.sum(top_values*topic_pagerank[top_indices],axis=1)


    # meilleurs documents :

    # doc qui ont les mots de la query sans pyterrier
    # df['text_lower'] = df['contenu'].str.lower() # Convert the text column to lowercase for case-insensitive search
    # query_words = q.lower().split() # Split the query into individual words
    # mask = df['text_lower'].str.contains('|'.join(query_words)) # boolean mask to filter rows containing any of the query words
    # doc_indices = df.index[mask] # indices des docs qui ont la query
    
    #res[doc_indices]+=100 # on leur donne plus de valeur pour qu'ils sortent d'abord

    #sorted_docs_index = np.argsort(-res)
    # best_docs = pd.read_csv(f'./Data/all.csv')["titre"].values[sorted_docs_index]

    best_docs = docs.sort_values(by="score", ascending=False)["titre"]
    return best_docs

In [67]:
# query = "affirmative action"
# print(sqd(query, topic_pagerank, D, terms_indexes, index)[:5])

8        Expressive therapies
11499        Apple and unions
12423           Shadow family
12384           Public sphere
12357    Gender mainstreaming
Name: titre, dtype: object


/var/folders/k2/pykvndls4t122_sjk529gvvc0000gn/T/ipykernel_15530/4254619777.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  docs["score"] = docs.apply(lambda x: sum([probas_c_q[Topic_names.index(topic)]*topic_pagerank[topic][x["id"]] for topic in Topic_names]), axis=1)


In [82]:
def score_normal_pagerank(q,normal_pagerank):
    
    # # doc qui ont les mots de la query (sans pyterrier)
    # df['text_lower'] = df['contenu'].str.lower() # Convert the text column to lowercase for case-insensitive search
    # query_words = q.lower().split() # Split the query into individual words
    # mask = df['text_lower'].str.contains('|'.join(query_words)) # boolean mask to filter rows containing any of the query words
    # doc_indices = df.index[mask] # indices des docs qui ont la query
    # normal_pagerank[doc_indices] +=100
    # 
    # sorted_docs_index = np.argsort(-normal_pagerank)
    # best_docs = pd.read_csv(f'./Data/all.csv')["titre"].values[sorted_docs_index]
    
    # Using a text index, recuperer les docs qui contiennent q
    br = pt.BatchRetrieve(index, wmodel="CoordinateMatch") # CoordinateMatch renvoie 1 si le terme est dans le doc, 0 sinon
    docs_retrieved = br.search(q) # Renvoie le nombre de termes de la query qui sont dans chaque doc
    docs_query = docs_retrieved[docs_retrieved["score"] == len(q.split())]  # Garder uniquement les docs qui contiennent tous les termes de la query
    sorted_index = docs_query["docno"].values
    
    df = pd.read_csv(f'./Data/all.csv')
    df.columns = ["id", "titre", "contenu", "liens"]
    
    docs = df[df["titre"].isin(sorted_index)]
    docs["score"] = normal_pagerank[docs["id"].values]
    
    best_docs = docs.sort_values(by="score", ascending=False)["titre"]
    
    return best_docs


# Experimental results

## Similarity ranking (4.1)

In [69]:
def OSim(t1, t2, n=20):
    # t1, t2 listes
    return len(set(t1[:n])&set(t2[:n]))/n

In [70]:
import itertools

def KSim(t1, t2):
    # t1, t2 listes
    U = set(t1)|set(t2)
    delta1 = U - set(t1)
    delta2 = U - set(t2)

    t1_prime = list(t1)+list(delta1)
    t2_prime = list(t2)+list(delta2)

    sim = 0
    for u,v in list(itertools.permutations(U, 2)):
        if np.sign(t1_prime.index(u)-t1_prime.index(v))==np.sign(t2_prime.index(u)-t2_prime.index(v)):
            sim+=1
    return sim/(len(U)*(len(U)-1))


## Tests on queries

In [20]:
queries = [ "affirmative action", "alcoholism", "amusement parks", 
            "architecture", "bicycling", "blues", "cheese", 
            "citrus groves", "classical guitar", "computer vision", 
            "cruises", "death valley", "field hockey", 
            "gardening", "graphic design", "gulf war", 
            "hiv", "java", "lipari",
            "lyme disease", "mutual funds", "national parks", 
            "parallel architecture", "recycling cans", "rock climbing", 
            "san francisco", "shakespeare", "stamp collecting", 
            "sushi", "table tennis", "telecommuting", 
            "vintage cars", "volcano", "zen buddhism", "zener"]

In [72]:
import numpy as np
import pickle

with open("terms_indexes.pkl", "rb") as f:
    terms_indexes = pickle.load(f)
with open("D.pkl", "rb") as f:
    D = pickle.load(f)
    for topic in Topic_names:
        D[topic] = np.reshape(D[topic],(D[topic].shape[1],1))
with open("topic_pagerank.pkl", "rb") as f:
    topic_pagerank = pickle.load(f)
    
index = pt.IndexFactory.of("./pd_index")

In [ ]:
for q in queries[:5]:
    print(q)
    print(sqd(q, topic_pagerank, D, terms_indexes, None)[:5])


affirmative action
['Art world' 'Cowles Art School' 'Cirebonese mask dance'
 'Silent Energy: New Art from China'
 'Public art of the Washington State Ferries system']
alcoholism
['Joel-Peter Witkin' 'Gandharvanpattu' 'Nowhereisland'
 'New Theatre Quarterly' 'Art']
amusement parks
['Jules Trobaugh' 'Society of Modern Women Artists' 'Tortillon'
 'Mediology' 'Suspension of disbelief']
architecture
['Leonardo (journal)' 'Narration' 'Screenplay' 'Narrative hook' 'Lath art']
bicycling
['Maternal mortality in fiction' 'Gerald Nordland' 'Film à clef'
 'After (art)' 'Corded quilting']


In [228]:
with open("normal_pagerank.pkl", "rb") as f:
        normal_pagerank = pickle.load(f)

In [236]:
for q in queries[:5]:
    print(q)
    print(score_normal_pagerank(q, normal_pagerank)[:5])

affirmative action
['Film à clef' 'Corded quilting' 'Ristra' 'Maternal mortality in fiction'
 'Gerald Nordland']
alcoholism
['Film à clef' 'Corded quilting' 'Ristra' 'Nowhereisland'
 'Maternal mortality in fiction']
amusement parks
['Film à clef' 'Corded quilting' 'Ristra' 'Maternal mortality in fiction'
 'Nowhereisland']
architecture
['Film à clef' 'Maternal mortality in fiction' 'Corded quilting'
 'Gerald Nordland' 'Ristra']
bicycling
['Film à clef' 'Maternal mortality in fiction' 'Corded quilting'
 'Gerald Nordland' 'Ristra']


In [234]:
for q in queries[:5]:
    print(q)
    print(score_normal_pagerank(q, topic_pagerank['business'])[:5])

affirmative action
['Artisan' 'Creative class' 'Andy Warhol' 'Self-fulfilling prophecy'
 'Performance art']
alcoholism
['Nowhereisland' 'New Theatre Quarterly' 'Gandharvanpattu' 'Artisan'
 'Creative class']
amusement parks
['Pseudohistory' 'Vignette (literature)' 'Maternal mortality in fiction'
 'After (art)' 'Tortillon']
architecture
['Maternal mortality in fiction' 'Gerald Nordland' 'Screenplay'
 'Pseudohistory' 'Leonardo (journal)']
bicycling
['Maternal mortality in fiction' 'Gerald Nordland' 'Screenplay'
 'Pseudohistory' 'Leonardo (journal)']


In [235]:
for q in queries[:5]:
    print(q)
    print(score_normal_pagerank(q, topic_pagerank['sports'])[:5])

affirmative action
['Film à clef' 'Corded quilting' 'Ristra' 'Maternal mortality in fiction'
 'After (art)']
alcoholism
['Film à clef' 'Corded quilting' 'Ristra' 'Maternal mortality in fiction'
 'After (art)']
amusement parks
['Maternal mortality in fiction' 'After (art)' 'Film à clef'
 'Corded quilting' 'Plot hole']
architecture
['Maternal mortality in fiction' 'Gerald Nordland' 'After (art)'
 'Film à clef' 'Corded quilting']
bicycling
['Maternal mortality in fiction' 'Gerald Nordland' 'After (art)'
 'Film à clef' 'Corded quilting']


## Similarité entre les ranking de pagerank normal et de pagerank avec topic

In [246]:
for q in queries:
    normal = score_normal_pagerank(q, normal_pagerank)
    topic = sqd(q, topic_pagerank, D, terms_indexes, None)
    print(q,"\tOSim:",OSim(normal, topic, n=20))
    #print("KSim,",KSim(normal[:20], topic[:20]))

affirmative action 	OSim: 0.05
alcoholism 	OSim: 0.05
amusement parks 	OSim: 0.2
architecture 	OSim: 0.15
bicycling 	OSim: 0.75
blues 	OSim: 0.3
cheese 	OSim: 0.25
citrus groves 	OSim: 0.05
classical guitar 	OSim: 0.15
computer vision 	OSim: 0.0
cruises 	OSim: 0.0
death valley 	OSim: 0.45
field hockey 	OSim: 0.1
gardening 	OSim: 0.0
graphic design 	OSim: 0.0
gulf war 	OSim: 0.2
hiv 	OSim: 0.1
java 	OSim: 0.0
lipari 	OSim: 0.0
lyme disease 	OSim: 0.05
mutual funds 	OSim: 0.0
national parks 	OSim: 0.0
parallel architecture 	OSim: 0.15
recycling cans 	OSim: 0.0
rock climbing 	OSim: 0.2
san francisco 	OSim: 0.1
shakespeare 	OSim: 0.2
stamp collecting 	OSim: 0.1
sushi 	OSim: 0.05
table tennis 	OSim: 0.0
telecommuting 	OSim: 0.0
vintage cars 	OSim: 0.15
volcano 	OSim: 0.0
zen buddhism 	OSim: 0.0
zener 	OSim: 0.0


In [43]:
for q in queries:
    normal = score_normal_pagerank(q, normal_pagerank)
    topic = sqd(q, topic_pagerank, D, terms_indexes, None)
    print(q,"\tKSim,",KSim(normal[:20], topic[:20]))

affirmative action 	KSim, 0.25384615384615383
alcoholism 	KSim, 0.5954415954415955
amusement parks 	KSim, 0.27692307692307694
architecture 	KSim, 0.22435897435897437
bicycling 	KSim, 0.30229419703103916
blues 	KSim, 0.38257575757575757
cheese 	KSim, 0.2807807807807808
citrus groves 	KSim, 0.16984126984126985
classical guitar 	KSim, 0.2822822822822823
computer vision 	KSim, 0.2076923076923077
cruises 	KSim, 0.2512820512820513
death valley 	KSim, 0.22948717948717948
field hockey 	KSim, 0.2739541160593792
gardening 	KSim, 0.30014224751066854
graphic design 	KSim, 0.28205128205128205
gulf war 	KSim, 0.2739541160593792
hiv 	KSim, 0.26720647773279355
java 	KSim, 0.267425320056899
lipari 	KSim, 0.24426450742240216
lyme disease 	KSim, 0.26180836707152494
mutual funds 	KSim, 0.24039829302987198
national parks 	KSim, 0.25889046941678523
parallel architecture 	KSim, 0.283072546230441
recycling cans 	KSim, 0.25775978407557354
rock climbing 	KSim, 0.2777777777777778
san francisco 	KSim, 0.286099865

### Comparaison des vecteurs PageRank (table 4)

In [46]:
pageranks = topic_pagerank.copy()
pageranks["normal"] = normal_pagerank

In [49]:
table2={}
for topic_1,topic_2 in list(itertools.permutations(["normal"]+Topic_names, 2)):
    print(topic_1,topic_2)
    sim = 0
    for q in queries:
        indexation1 = score_normal_pagerank(q, pageranks[topic_1])
        indexation2 = score_normal_pagerank(q, pageranks[topic_2])
        sim += KSim(indexation1[:20], indexation2[:20])
    sim /= len(queries)
    table2 = {f"{topic_1}/{topic_2}": sim}

    

normal arts
normal games
